In [6]:
import pandas as pd
import os
import datetime
import time
import random

In [7]:
pd.set_option("display.max_columns", None)

In [33]:
data = {}
param = {}
input_directory = '../temp'
param_directory = '../param'
for _filename in os.listdir(input_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(input_directory+'/'+_filename,  header = None)
    filetype = _filename.split('-')[1]
    column_dict = { 
            'sked': ['from', 'to', 'al','fln', 'actype', 'depday','dep','arrday','arr'],
            'comp': ['from', 'to', 'al', 'fln','actype', 'depday','dep','arrday','arr'],
            'demand': ['orig', 'dest', 'volume','weight', 'rev'],
            'demand_curve': ['orig', 'dest', 'ttt_1','ttt_2', 'ttt_3','ttt_4'],          
            'route_cost': ['from','to', 'actype', 'value'],
            'airport_cost': ['ap', 'actype', 'value'],
            'config': ['actype', 'cap_1', 'cap_2']
    }
    df_data.columns = column_dict[filetype]
    data[filetype] = df_data
    
for _filename in os.listdir(param_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(param_directory+'/'+_filename,  header = None)
    column_dict = { 
            'network' : ['stops_allowed'],
            'connections': ['hub', 'minct', 'maxct'],
            'preferences' : ['from', 'to', 'stop_penalty']
    }
    df_data.columns = column_dict[_filename]
    param[_filename] = df_data

test-airport_cost
test-comp
test-config
test-demand
test-demand_curve
test-route_cost
test-sked
connections
preferences


In [116]:
#inputs
your_sked = data['sked']
comp_sked = data['comp']
param_connect = param['connections']
param_pref = param['preferences']
param_pref['od'] = param_pref["from"]+param_pref["to"]
max_stop = 2



full_sked = pd.concat([your_sked,comp_sked])
full_sked["dep"] = pd.to_datetime(full_sked["dep"]) + full_sked["depday"]*datetime.timedelta(days=1)
full_sked["arr"] = pd.to_datetime(full_sked["arr"]) + full_sked["arrday"]*datetime.timedelta(days=1)
full_sked["id"] = full_sked['al'] + full_sked['fln'].astype('str')


param_connect['minct'] = pd.to_timedelta(param_connect['minct'])
param_connect['maxct'] = pd.to_timedelta(param_connect['maxct'])

df_connect = full_sked.add_suffix('_1').reset_index(drop=True)
df_connect['nbstops'] = 0

list_itin = {}
list_itin[0] = df_connect


#build itineraries
for i in range (1,max_stop+1):
    cnx_p = param_connect.copy()
    itins = list_itin[i-1].merge(full_sked.add_suffix('_'+str(i+1)), how= "cross")
    itins['nbstops'] = i
    
    itins['cnx_time_'+str(i)] = itins['dep_'+str(i+1)] - itins['arr_'+str(i)]
    
    cnx_p.columns = ['to_'+str(i), 'minct_'+str(i), 'maxct_'+str(i)]
    
    #filter connections
    itins.drop(itins[itins['to_'+str(i)] != itins['from_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(i)] == itins['to_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(1)] == itins['to_'+str(i+1)]].index, inplace=True)
    
    itins = itins.merge(cnx_p, on = ['to_'+str(i)])
    
    itins.drop(itins[itins['cnx_time_'+str(i)] < itins['minct_'+str(i)]].index, inplace=True)
    itins.drop(itins[itins['cnx_time_'+str(i)] > itins['maxct_'+str(i)]].index, inplace=True)
   
    list_itin[i] = itins



In [117]:
list_itin_summary = pd.DataFrame(columns=["id","index", "od","travel_time", "nbstops","score"])


#add attributes
for i in range(0,max_stop+1):
    list_itin[i]["travel_time"] = list_itin[i]["arr_"+str(i+1)]-list_itin[i]["dep_1"]
    list_itin[i]["od"] = list_itin[i]["from_1"]+list_itin[i]["to_"+str(i+1)]
    list_itin[i]["index"] = list_itin[i].index
    list_itin[i]['id'] = list_itin[i]["nbstops"].astype(str)+"-"+list_itin[i]["index"].astype(str)
    list_itin[i] = list_itin[i].merge(param_pref, on="od")

    #calculate score
    list_itin[i]["score"]= list_itin[i]["travel_time"].dt.seconds+list_itin[i]["nbstops"]*list_itin[i]["stop_penalty"]*3600

    #prepare the data
    list_itin_summary= pd.concat([list_itin_summary,list_itin[i][["id","index", "od","travel_time", "nbstops","score"]]]) 




In [118]:
od_itin = {}
#organize data by OD
for od in param_pref.od.unique():  
    
    od_itin[od] = list_itin_summary[list_itin_summary['od']==od].copy()

In [119]:
#variables
timep = 4

#define user arrivals
demand_rand = {}
demand_curve = data['demand_curve']
demand = data['demand']

demand_by_ttt = pd.merge(demand, demand_curve, on=["orig","dest"])

for i in range(1,1+timep):
    demand_by_ttt['d_ttt_'+str(i)]=(demand_by_ttt['ttt_'+str(i)]*demand_by_ttt['volume']).astype('int')

#shuffle pax

for i in range(1,1+timep):
    randm = []
    for indx, row in demand_by_ttt.iterrows():
        for k in range(1,row['d_ttt_'+str(i)]):
            randm.append(row['orig']+row['dest'])
    random.shuffle(randm)
    demand_rand[i] = randm
    

In [122]:
def choose_path(list_pax, itineraries):
    #choose an itin randomly among available ones
    for od_pax in list_pax:
        if len(itineraries[od_pax])>0 :
            choice = random.choices(list(itineraries[od_pax]["id"]), weights = list(itineraries[od_pax]["score"]), k=1)
            stop_index = choice[0].split('-')
            #remove capacity from itinerary
    return 1

choose_path(demand_rand[1], od_itin)

1

In [115]:
list_itin_summary[list_itin_summary['od']=='DUSTXL']

,id,index,od,travel_time,nbstops,score
0,0-0,0,DUSTXL,0 days 01:32:00,0,5520
1,14-0,14,DUSTXL,0 days 02:00:00,0,7200
0,0-1,0,DUSTXL,0 days 05:01:00,1,21660
1,7-1,7,DUSTXL,0 days 05:15:00,1,22500
